In [3]:
import os
import sys
import pandas as pd
import gzip

from computage.utils.data_utils import download_meta, download_dataset

In [4]:
datasets_needed = ['GSE56046', 'GSE56581', 'GSE62867', 'GSE69138', 
             'GSE107143', 'GSE203399', 'GSE32148', 'GSE53840',
              'GSE53841', 'GSE67705', 'GSE67751', 'GSE77696', 
              'GSE81961', 'GSE87640', 'GSE87648', 'GSE100264', 
              'GSE107080', 'GSE117859', 'GSE117860', 'GSE140800', 
              'GSE143942', 'GSE145714', 'GSE185389', 'GSE185390', 
              'GSE217633', 'GSE49909', 'GSE56606', 'GSE62003', 
              'GSE131461', 'GSE166611', 'GSE193836', 'GSE42861', 
              'GSE71841', 'GSE99624', 'GSE131989', 'GSE134429', 
              'GSE137593', 'GSE137594', 'GSE138653', 'GSE175364', 
              'GSE176168', 'GSE228104', 'GSE43976', 'GSE59685', 
              'GSE72774', 'GSE72776', 'GSE103929', 'GSE106648', 
              'GSE111223', 'GSE111629', 'GSE112596', 'GSE122244', 
              'GSE130029', 'GSE130030', 'GSE130491', 'GSE144858', 
              'GSE151355', 'GSE156994', 'GSE219293', 'GSE72338', 
              'GSE118468', 'GSE118469', 'GSE145714', 'GSE131752', 
              'GSE182991', 'GSE214297']

datasets_present = ['GSE107143', 'GSE109096', 'GSE157131', 'GSE197670',
                     'GSE203399', 'GSE222595', 'GSE32148', 'GSE38291', 
                     'GSE42861', 'GSE46394', 'GSE48325', 'GSE49909', 
                     'GSE50498', 'GSE52588', 'GSE53840', 'GSE56046', 
                     'GSE56581', 'GSE59685', 'GSE61256', 'GSE62003', 
                     'GSE62867', 'GSE69138', 'GSE73103', 'GSE80970', 
                     'GSE84395', 'GSE87640', 'GSE87648', 'GSE132203', 
                     'GSE30870', 'GSE36054', 'GSE40279', 'GSE41169', 
                     'GSE51032', 'GSE55763', 'GSE64495', 'GSE69270', 
                     'GSE72775', 'GSE87571']

datasets_load = list(set(datasets_needed)&set(datasets_present))
print(datasets_load)
print(len(datasets_load))

['GSE203399', 'GSE87648', 'GSE32148', 'GSE59685', 'GSE53840', 'GSE42861', 'GSE69138', 'GSE62867', 'GSE56581', 'GSE62003', 'GSE87640', 'GSE56046', 'GSE49909', 'GSE107143']
14


## Variant 1. Downloading and saving table with all data

In [3]:
dir = "computage/temp_loaded_files"
tissue = 'Blood'
df_tissue = pd.DataFrame()
df_tissue_meta = pd.DataFrame()

for filename in datasets_load:
    #loading
    meta = download_meta('./meta_table_datasets.xlsx')
    download_dataset(meta, filename, dir)

    #reading -> saving
    filepath_pklgz = dir + '/' + filename + '.pkl.gz'
    filepath_pkl = dir + '/' + filename + '.pkl'
    
    with gzip.open(filepath_pklgz,'rb')as f_in:
        with open(filepath_pkl, 'wb') as f_out:
            f_out.write(f_in.read())

    df = pd.read_pickle(filepath_pkl)

    #reading pkl
    meta_data = df.get('meta')
    data = df.get('data')
    print(filename, 'dataframe df was created:', ' data:', data.shape, ' meta_data:', meta_data.shape) ##

    #filtering, creating data_tissue, meta_data_tissue
    meta_data_tissue = meta_data[(meta_data['Condition'] == 'HC')&(meta_data['Tissue'] == tissue)]
    # meta_data_tissue.loc[:, 'SourceGSE'] = filename
    meta_data_tissue['SourceGSE'] = filename
    data_tissue = data.loc[data.index.isin(meta_data_tissue.index)]
    print(filename, 'dataframe was filtered', 'rows:', data_tissue.shape[0]) ##

    #saving tmp_csvs
    data_tissue.to_csv(dir + '/temp_csv/' + filename + '.csv')
    meta_data_tissue.to_csv(dir + '/temp_csv/' + filename +'_meta' + '.csv')
    print(filepath_pkl, 'is saved')

    #merging
    df_tissue = pd.concat([df_tissue, data_tissue])
    df_tissue_meta = pd.concat([df_tissue_meta, meta_data_tissue])


Dataset GSE59685 saved to computage/temp_loaded_files
GSE59685 dataframe df was created:  data: (531, 485577)  meta_data: (531, 10)
GSE59685 dataframe was filtered rows: 9


/tmp/ipykernel_14420/3913735079.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_data_tissue['SourceGSE'] = filename


computage/temp_loaded_files/GSE59685.pkl is saved
Dataset GSE69138 saved to computage/temp_loaded_files
GSE69138 dataframe df was created:  data: (185, 456344)  meta_data: (185, 8)
GSE69138 dataframe was filtered rows: 0
computage/temp_loaded_files/GSE69138.pkl is saved
Dataset GSE107143 saved to computage/temp_loaded_files
GSE107143 dataframe df was created:  data: (16, 460295)  meta_data: (16, 7)
GSE107143 dataframe was filtered rows: 8


/tmp/ipykernel_14420/3913735079.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_data_tissue['SourceGSE'] = filename


computage/temp_loaded_files/GSE107143.pkl is saved
Dataset GSE56046 saved to computage/temp_loaded_files


: 

## Variant 2 (step by step). Downloading and saving table with all data

### Downloading, saving to csv: metadata, data

In [4]:
dir = "computage/temp_loaded_files"
tissue = 'Blood'
counter = 0

for filename in datasets_load:
    print('#######')
    print("FILE", counter)
    print('#######')
    counter += 1

    #loading
    meta = download_meta('./meta_table_datasets.xlsx')
    download_dataset(meta, filename, dir)

    #reading -> saving
    filepath_pklgz = dir + '/' + filename + '.pkl.gz'
    filepath_pkl = dir + '/' + filename + '.pkl'
    
    with gzip.open(filepath_pklgz,'rb')as f_in:
        with open(filepath_pkl, 'wb') as f_out:
            f_out.write(f_in.read())

    df = pd.read_pickle(filepath_pkl)

    #reading pkl
    meta_data = df.get('meta')
    data = df.get('data')
    print(filename, 'dataframe df was created:', ' data:', data.shape, ' meta_data:', meta_data.shape) ##

    #filtering, creating data_tissue, meta_data_tissue
    meta_data_tissue = meta_data[(meta_data['Condition'] == 'HC')&(meta_data['Tissue'] == tissue)]
    meta_data_tissue['SourceGSE'] = filename
    data_tissue = data.loc[data.index.isin(meta_data_tissue.index)]
    print(filename, 'dataframe was filtered', 'rows:', data_tissue.shape[0]) ##

    #saving tmp_csvs
    data_tissue.to_csv(dir + '/temp_csv/' + filename + '.csv')
    meta_data_tissue.to_csv(dir + '/temp_csv/' + filename +'_meta' + '.csv')
    print(filepath_pkl, 'is saved')

###############################
FILE 11
###############################
Dataset GSE87640 saved to computage/temp_loaded_files
GSE87640 dataframe df was created:  data: (240, 485577)  meta_data: (240, 10)
GSE87640 dataframe was filtered rows: 84


/tmp/ipykernel_17540/2067655150.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_data_tissue['SourceGSE'] = filename


computage/temp_loaded_files/GSE87640.pkl is saved
###############################
FILE 12
###############################
Dataset GSE53840 saved to computage/temp_loaded_files
GSE53840 dataframe df was created:  data: (120, 485577)  meta_data: (120, 26)
GSE53840 dataframe was filtered rows: 0
computage/temp_loaded_files/GSE53840.pkl is saved
###############################
FILE 13
###############################
Dataset GSE87648 saved to computage/temp_loaded_files
GSE87648 dataframe df was created:  data: (382, 460398)  meta_data: (382, 10)
GSE87648 dataframe was filtered rows: 178


/tmp/ipykernel_17540/2067655150.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_data_tissue['SourceGSE'] = filename


computage/temp_loaded_files/GSE87648.pkl is saved
###############################
FILE 14
###############################


: 

### Creation of table

In [16]:
dir = "computage/temp_loaded_files"
dir_csv = dir + '/temp_csv/'
table_prep = pd.DataFrame()
column_names = []

for filename in datasets_load:

    with open(dir_csv + filename + '_meta.csv','r') as file:
        header = file.readline().split(sep=',')
        column_names.extend(header)

column_names = sorted(list(set(column_names)))

# Создаем пустую таблицу со всеми колонками
table_prep =  pd.DataFrame(column_names).T
table_prep.columns = table_prep.iloc[0,:]
table_prep = table_prep.drop(table_prep.index)
table_prep


full_table_tissue_name = 'full_blood_HC'
table_prep.to_csv('computage/' + full_table_tissue_name + '.csv')

In [17]:
for filename in datasets_load:
    meta_data_tissue = pd.read_csv(dir_csv + filename + '_meta.csv')
    full_meta_tissue = pd.concat([table_prep, meta_data_tissue])
    full_meta_tissue = full_meta_tissue.set_index('Unnamed: 0')
    full_meta_tissue.to_csv('computage/' + full_table_tissue_name + '.csv', header=False, mode='a')

/tmp/ipykernel_2087/510909243.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_meta_tissue = pd.concat([table_prep, meta_data_tissue])
/tmp/ipykernel_2087/510909243.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_meta_tissue = pd.concat([table_prep, meta_data_tissue])
/tmp/ipykernel_2087/510909243.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when d

теперь для данных

In [5]:
dir = "computage/temp_loaded_files"
dir_csv = dir + '/temp_csv/'
table_prep = pd.DataFrame()
column_names = []


for filename in datasets_load:

    with open(dir_csv + filename + '.csv','r') as file:
        header = file.readline().split(sep=',')
        column_names.extend(header)

column_names = list(set(column_names))

# Создаем пустую таблицу со всеми колонками
table_prep =  pd.DataFrame(column_names).T
table_prep.columns = table_prep.iloc[0,:]
table_prep = table_prep.drop(table_prep.index)
table_prep


full_table_tissue_name = 'full_blood_HC_CpGs'
table_prep.to_csv('computage/' + full_table_tissue_name + '.csv')

In [6]:
len(table_prep.columns.unique())

487183

In [7]:
for filename in datasets_load:
    data_tissue = pd.read_csv(dir_csv + filename + '.csv')
    full_meta_tissue = pd.concat([table_prep, data_tissue])
    full_meta_tissue = full_meta_tissue.set_index('Unnamed: 0')
    full_meta_tissue.to_csv('computage/' + full_table_tissue_name + '.csv', header=False, mode='a')
    break

KeyboardInterrupt: 